<a href="https://colab.research.google.com/github/razmik/financial_statement_extractor/blob/main/Coles_Statement_Reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 24.8 MB/s eta 0:00:00


### Upload and unzip the ZIP file

Compress all the statement files into a zip file and upload to the Colab Files Environment.  
Once uploaded, run the below command.

In [ ]:
# Assuming the uploaded file is 'example.zip'
zip_file_name = 'Archive.zip'

# Create a directory to unzip the file into
output_dir = 'unzipped_folder'
!mkdir -p {output_dir}

# Unzip the file into the specified directory
!unzip -q {zip_file_name} -d {output_dir}

# List the contents of the unzipped directory to verify
!ls {output_dir}

In [ ]:
import os
from tqdm import tqdm
import fitz  # PyMuPDF
import pandas as pd
import re
import calendar
import warnings
warnings.filterwarnings('ignore')

def extract_transactions_from_pdf(pdf_path):
    # Open the document
    document = fitz.open(pdf_path)

    # Initialize lists to store transactions
    transactions = []

    # Iterate through the pages starting from page 2
    for page_num in range(1, len(document)):
        page = document.load_page(page_num)
        text = page.get_text("text")

        # Regular expression pattern to match the transaction entry
        pattern = re.compile(r'(\w{3} \d{2}\n.+?\n-?\d+\.\d{2}\n\d{14,})\n')

        # Find all matches
        matches = pattern.findall(text)

        # Convert each match into an array of strings
        transaction_lines = [match.split('\n') for match in matches]

        # Append to transactions list
        transactions.extend(transaction_lines)

    # Create a DataFrame
    df = pd.DataFrame(transactions, columns=['Date', 'Description', 'Amount', 'Reference Number'])

    return df

def save_transactions_to_csv(df, output_csv_path):
    df.to_csv(output_csv_path, index=False)

if __name__ == "__main__":

    output_csv_path = 'transactions.csv'  # Path to save the CSV file

    # List files in the directory
    file_list = os.listdir(output_dir)

    transactions_df_list = []

    # Print each filename
    for filename in tqdm(file_list):

      if not filename.lower().endswith('.pdf'):
        continue

      pdf_path = f"{output_dir}/{filename}"

      # Extract transactions and save to CSV
      transactions_df = extract_transactions_from_pdf(pdf_path)
      # Append year_month
      transactions_df['Statement_Period'] = filename.split('.pdf')[0].split('_')[-1]
      transactions_df['statement_year'] = transactions_df['Statement_Period'].str[-4:]
      transactions_df['statement_month'] = transactions_df['Statement_Period'].str[1:3]
      transactions_df['statement_year_month'] = transactions_df['statement_year'] + transactions_df['statement_month']
      transactions_df['statement_month_name'] = transactions_df['statement_month'].apply(lambda x: calendar.month_name[int(x)])

      transactions_df_list.append(transactions_df)

    # Save the output df
    combined_df = pd.concat(transactions_df_list, ignore_index=True)
    save_transactions_to_csv(combined_df, output_csv_path)
    print(f"Transactions have been successfully saved to {output_csv_path}")


100%|██████████| 27/27 [00:00<00:00, 59.84it/s]

Transactions have been successfully saved to transactions.csv
